In [8]:
import plotly
from plotly.graph_objs import Scatter, Layout
import numpy as np
import tensorflow as tf
import sys
plotly.offline.init_notebook_mode(connected=True)
import IPython.display

In [9]:
sample_length = 50001
time_per_sample = 0.01
signal_time = np.linspace(num=sample_length,start = 0, stop = sample_length * time_per_sample )
signal_amp = np.sin(signal_time*2*np.pi) + np.random.normal(size=sample_length)*0.02
    #np.sin(2+signal_time*1.7*np.pi)*0.5 + \
    #np.sin(1+signal_time*2.2*np.pi) + \
    

In [10]:
s_i = 0
e_i = s_i + 100
x = plotly.offline.iplot({
    "data": [Scatter(x=signal_time[s_i:e_i],y=signal_amp[s_i:e_i])],
    "layout": Layout(title="")
    
})



In [11]:
sequence_length = 50
prediction_length = 1
input_feature_count = 1
output_feature_count = 1
batch_size = 128 #512
hidden_count_per_layer = [16,16,16]

tf.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, sequence_length, input_feature_count], name = 'inputs')
targets = tf.placeholder(tf.float32, [None, output_feature_count], name = 'targets')
keep_prob = tf.placeholder(tf.float32, name = 'keep')
learning_rate = tf.placeholder(tf.float32, name = 'learning_rate')

In [12]:
layers = []


for hidden_count in hidden_count_per_layer:
    layer =  tf.nn.rnn_cell.LSTMCell(hidden_count, state_is_tuple=True)
    layer_with_dropout = tf.nn.rnn_cell.DropoutWrapper(layer,
                                          input_keep_prob=keep_prob,
                                          output_keep_prob=1.0)
    layers.append(layer)
hidden_network = tf.nn.rnn_cell.MultiRNNCell(layers, state_is_tuple=True)   



In [32]:
def pack_state_tupel(state_tupel):
    print(state_tupel)
    if isinstance(state_tupel, tf.Tensor) or not hasattr(state_tupel, '__iter__'):
        return state_tupel
    else:
        l = []
        for item in state_tupel:
            print("in = ",item)
            v = pack_state_tupel(item)
            print("po = ",v)
            l.append(v)
            
        return tf.concat(1, l)
        #return tf.concat(1, [pack_state_tupel(item) for item in state_tupel])

def unpack_state_tupel(state_tensor, sizes):
    def _unpack_state_tupel(state_tensor_, sizes_, offset_):
        if isinstance(sizes_, tf.Tensor) or not hasattr(sizes_, '__iter__'): 
            return tf.reshape(state_tensor_[:, offset_ : offset_ + sizes_], (-1, sizes_)), offset_ + sizes_
        else:
            result = []
            for size in sizes_:
                s, offset_ = _unpack_state_tupel(state_tensor_, size, offset_)
                result.append(s)
            if isinstance(sizes_, tf.nn.rnn_cell.LSTMStateTuple):
                return tf.nn.rnn_cell.LSTMStateTuple(*result), offset_
            else:
                return tuple(result), offset_
    return _unpack_state_tupel(state_tensor, sizes, 0)[0]

In [33]:
print("states in network", hidden_network.state_size)

zero_state = pack_state_tupel(hidden_network.zero_state(batch_size, tf.float32))
initial_state = tf.placeholder_with_default(zero_state, None, name = "initial_state")
#state = unpack_state_tupel(self.input_state, rnn.state_size)

#zero_state = pack_state_tuple(hidden_network.zero_state(batch_size, tf.float32))
#initial_state = tf.placeholder_with_default(
#    zero_state, 
#    (sz,1), 
#    name="initial_state")



states in network (LSTMStateTuple(c=16, h=16), LSTMStateTuple(c=16, h=16), LSTMStateTuple(c=16, h=16))
(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState/zeros:0' shape=(128, 16) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState/zeros_1:0' shape=(128, 16) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState_1/zeros:0' shape=(128, 16) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState_1/zeros_1:0' shape=(128, 16) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState_2/zeros:0' shape=(128, 16) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState_2/zeros_1:0' shape=(128, 16) dtype=float32>))
in =  LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState/zeros:0' shape=(128, 16) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState_9/LSTMCellZeroState/zeros_1:0' shape=(128, 16) dtype=float32>)
LSTMStateTuple(c=<tf.Tensor 'MultiR

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.

In [29]:
# Test to check that the pack and unpack functions are eachothers inverse

#copy of the input (packed) shape (x,1)
initial_copy = tf.slice(initial_state,(0,0),(sz,1))

#unpack into (LSTMTuple(),LSTMTuple(),....)
unpacked = unpack_state_tuple(hidden_network,batch_size,initial_copy)
rnn_tuple_state = unpacked
#repack into vector (x,1)
packed_again = pack_state_tuple(unpacked)


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    #v_outputs, v_state = sess.run([outputs,state], feed_dict={inputs: batch_inputs, targets: batch_targets})
    cp = sess.run(initial_copy,  feed_dict={}) #initial_state: initial_state_input})
    #cp[0] = 1
    cp[:,0] = np.linspace(start=0,stop=cp.shape[0],num=cp.shape[0])
    
    cp,up,cp2 = sess.run([initial_copy,unpacked,packed_again],  feed_dict={initial_state: cp}) #initial_state: initial_state_input})
    #print("cp",cp.shape,cp)
    #print("up",unpacked,up)
    #print("cp2",packed_again,cp2)
    #print("up2",unpacked_again,up2)
    diff = cp - cp2
    print("diff",np.sum(diff))

diff 0.0


In [30]:
#out_weights=tf.Variable(tf.random_normal([hidden_count_per_layer[-1],output_feature_count]))
#out_bias=tf.Variable(tf.random_normal([output_feature_count]))
print("inputs ",inputs.shape)
outputs, state = tf.nn.dynamic_rnn(hidden_network, inputs, dtype=tf.float32, initial_state=rnn_tuple_state, )
state_packed = pack_state_tuple(state)
print("outputs before transpose", outputs.shape)
outputs = tf.transpose(outputs, [1, 0, 2])
print("outputs after transpose", outputs.shape)
last_output = tf.gather(outputs, int(outputs.get_shape()[0]) - 1)
print("last output", last_output.shape)
                                   
#out_size = target.get_shape()[2].value
predictions = tf.contrib.layers.fully_connected(last_output, output_feature_count, activation_fn=None)
print("prediction", predictions.shape)
print("targets", targets.shape)
#prediction = tf.nn.softmax(logit)
#loss = tf.losses.softmax_cross_entropy(target, logit)


inputs  (?, 50, 1)
outputs before transpose (128, 50, 16)
outputs after transpose (50, 128, 16)
last output (128, 16)
prediction (128, 1)
targets (?, 1)


In [12]:
loss = tf.reduce_sum(tf.squared_difference(predictions, targets))

#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [13]:
start_indices = np.linspace(
    0,
    sample_length-sequence_length-prediction_length-1,
    sample_length-sequence_length-prediction_length-1, dtype= np.int32)

dev_size_perc = 0.20
test_size_perc = 0.20

dev_size = int(np.floor(start_indices.shape[0] * dev_size_perc))
test_size  = int(np.floor(start_indices.shape[0] * test_size_perc))
train_size = start_indices.shape[0] - test_size - dev_size
train_batch_count = int(np.floor(train_size / batch_size))
dev_batch_count = int(np.floor(dev_size / batch_size))
test_batch_count = int(np.floor(test_size / batch_size))

print("dataset size %d" %(start_indices.shape[0]))
print("%d Examples (%d batches) in train set" %(train_size, train_batch_count))
print("%d Examples (%d batches) in dev set" %(dev_size,dev_batch_count))
print("%d Examples (%d batches) in test set" %(test_size,test_batch_count))



dataset size 49949
29971 Examples (234 batches) in train set
9989 Examples (78 batches) in dev set
9989 Examples (78 batches) in test set


In [14]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]

def get_batch(batch_index, indexes, size=batch_size):
    batch_start_indexes = indexes[batch_index*size:batch_index*size+size]
    batch_inputs = np.zeros((size,sequence_length, input_feature_count))
    batch_targets = np.zeros((size,prediction_length))
    for i in range(size):
        se = batch_start_indexes[i]
        part = signal_amp[se:se+sequence_length]
        batch_inputs[i,0:sequence_length,0] = part
        batch_targets[i,0] = signal_amp[se+sequence_length+1]

    return batch_inputs,batch_targets

batch_inputs,batch_targets = get_batch(train_batch_count-1,train_indices)
print(batch_inputs.shape,batch_targets.shape)

example_inputs = batch_inputs[0,:,:]
example_targets =  batch_targets[0,:]
print(example_inputs.shape)
#b_i = 1
#b_s = batch_inputs[b_i,0:sequence_length,0]
#plotly.offline.iplot({
#    "data": [Scatter(y=b_s)],
#    "layout": Layout(title="")
#})

(128, 50, 1) (128, 1)
(50, 1)


In [ ]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    np.random.shuffle (train_indices)
    
    batch_inputs,batch_targets = get_batch(0, train_indices)
    print("batch input shape", batch_inputs.shape)
    #v_outputs, v_state = sess.run([outputs,state], feed_dict={inputs: batch_inputs, targets: batch_targets})
    v_predictions, v_state, v_state_packed = sess.run([predictions,state, state_packed], 
                                      feed_dict={
                                          inputs: batch_inputs, 
                                          targets: batch_targets
                                      })
    print(v_predictions.shape)
    print(v_predictions[0],batch_targets[0])
    for i in range(0,120):
        v_predictions, v_outputs, v_state, v_state_packed, v_loss, v_opt = sess.run(
            [predictions, outputs, state, state_packed, loss, opt], 
            feed_dict={
                learning_rate: 0.02, 
                inputs: batch_inputs, 
                targets: batch_targets,
                initial_state: v_state_packed
            }) #})
        print(v_loss,v_predictions[0],batch_targets[0])
 

    
    

In [15]:
np.random.shuffle (start_indices)
train_indices = start_indices[0:int(train_size)]
dev_indices= start_indices[int(train_size):int(train_size+dev_size)]
test_indices = start_indices[int(train_size+dev_size):int(train_size+dev_size+test_size)]

epoch_count = 5

loss_results = np.zeros((epoch_count,2))

def get_dev_loss():
    epoch_dev_loss = 0.0
    for devi in range(dev_batch_count):
        batch_inputs,batch_targets = get_batch(devi, dev_indices)

        batch_dev_loss = sess.run(loss,feed_dict={inputs:batch_inputs,targets:batch_targets})
        print("  Dev results batch %d, loss %s" %(  devi, str(batch_dev_loss)))  

        epoch_dev_loss += batch_dev_loss
        #sys.stdout.write('.')
        #sys.stdout.flush()
    return epoch_dev_loss / dev_size

def generate_graph(graph_size=200):
    prime_size = 20
    
    prime_signal_start_i = 0
    
    tmp_signal = np.zeros((graph_size,1))
    tmp_signal[0:prime_size,0] = signal_amp[prime_signal_start_i:(prime_signal_start_i+prime_size)]
    #tmp_signal[0:prime_size,0] = np.random.normal(size=prime_size)*0.6+0.1
    tmp_batch = np.zeros((1,sequence_length,1))
    
    _state_packed = None
    for end in range(prime_size, graph_size):
        #end = prime_size
        tmp_batch[0,:,0] = tmp_signal.take(range((end-sequence_length),end), mode='wrap')
        if _state_packed is None:
            _state_packed , _prediction = sess.run(
                [state_packed, predictions[0,0]], 
                feed_dict={learning_rate: 0.02, inputs: tmp_batch})
        else:
            _state_packed , _prediction = sess.run(
                [state_packed, predictions[0,0]], 
                feed_dict={learning_rate: 0.02, initial_state: _state_packed, inputs: tmp_batch})
            
        #print(_prediction)
        tmp_signal[end,0] = _prediction
        sys.stdout.write('.')
        sys.stdout.flush()
    print("")
    plotly.offline.iplot({
       "data": [Scatter(y=tmp_signal[:,0])],
       "layout": Layout(title="")})


init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    epoch_dev_loss = get_dev_loss()    

    #print("")            
    print("Dev results epoch start, loss %s" %(  str(epoch_dev_loss),))  

    for epoch in range(0,epoch_count):
        np.random.shuffle (train_indices)
        epoch_train_loss = 0.0
        for ti in range(train_batch_count):
            batch_inputs,batch_targets = get_batch(ti, train_indices)

            batch_train_loss, _ = sess.run([loss, opt], feed_dict={learning_rate: 0.002, inputs: batch_inputs, targets: batch_targets})
            print("  Train results batch %d, loss %s" %(  ti, str(batch_train_loss)))  
            epoch_train_loss += batch_train_loss
            #sys.stdout.write('.')
            #sys.stdout.flush()
        #print("")
        epoch_train_loss = epoch_train_loss / train_size
        print("Training results epoch %d, loss %s" %( epoch, str(epoch_train_loss)))
        epoch_dev_loss = get_dev_loss()    
        #print("")            
        print("Dev results epoch %d, loss %s" %( epoch, str(epoch_dev_loss)))  
        loss_results[epoch,0] = epoch_train_loss
        loss_results[epoch,1] = epoch_dev_loss
        ti += 1
        generate_graph()
    generate_graph(graph_size=1000)
        

  Dev results batch 0, loss 64.349
  Dev results batch 1, loss 64.1024
  Dev results batch 2, loss 60.566
  Dev results batch 3, loss 62.7298
  Dev results batch 4, loss 60.85
  Dev results batch 5, loss 58.9895
  Dev results batch 6, loss 57.6051
  Dev results batch 7, loss 70.635
  Dev results batch 8, loss 63.5203
  Dev results batch 9, loss 59.1915
  Dev results batch 10, loss 62.316
  Dev results batch 11, loss 68.2039
  Dev results batch 12, loss 65.3623
  Dev results batch 13, loss 57.5328
  Dev results batch 14, loss 57.3198
  Dev results batch 15, loss 61.9743
  Dev results batch 16, loss 59.1721
  Dev results batch 17, loss 66.7652
  Dev results batch 18, loss 61.991
  Dev results batch 19, loss 62.4706
  Dev results batch 20, loss 66.4418
  Dev results batch 21, loss 63.6695
  Dev results batch 22, loss 61.8375
  Dev results batch 23, loss 55.5109
  Dev results batch 24, loss 59.9885
  Dev results batch 25, loss 62.4615
  Dev results batch 26, loss 57.8135
  Dev results batc

  Train results batch 132, loss 0.0565818
  Train results batch 133, loss 0.0580997
  Train results batch 134, loss 0.0700472
  Train results batch 135, loss 0.0697948
  Train results batch 136, loss 0.0695701
  Train results batch 137, loss 0.0610017
  Train results batch 138, loss 0.0691938
  Train results batch 139, loss 0.0648346
  Train results batch 140, loss 0.0615873
  Train results batch 141, loss 0.0634703
  Train results batch 142, loss 0.0643701
  Train results batch 143, loss 0.0609675
  Train results batch 144, loss 0.068244
  Train results batch 145, loss 0.0717636
  Train results batch 146, loss 0.0482297
  Train results batch 147, loss 0.0550336
  Train results batch 148, loss 0.0626072
  Train results batch 149, loss 0.0519962
  Train results batch 150, loss 0.0757212
  Train results batch 151, loss 0.0599381
  Train results batch 152, loss 0.0645147
  Train results batch 153, loss 0.0609304
  Train results batch 154, loss 0.061447
  Train results batch 155, loss 0.05

InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,1] vs. shape[1] = [128,16]
	 [[Node: rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/lstm_cell/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](rnn/while/TensorArrayReadV3, rnn/while/Switch_3:1, rnn/while/rnn/multi_rnn_cell/cell_2/cell_2/lstm_cell/split/split_dim)]]

Caused by op 'rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/lstm_cell/concat', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 